In [2]:
import pandas as pd 
import numpy as np 
import talib as tb
import yfinance as yf

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
df_eth = pd.read_csv(r'../data/cryptocsv/eth.csv')

# Creating Indicators

In [3]:
df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-04-26', 
                      progress=False,
)

df_dollar = yf.download('DX=F', 
                      start='2017-11-09', 
                      end='2022-04-26', 
                      progress=False,
)
df_dollar.head(10)


1 Failed download:
- ^GSPC: No data found for this date range, symbol may be delisted

1 Failed download:
- DX=F: No data found for this date range, symbol may be delisted


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [4]:
df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)


In [5]:
df_eth[~(df_eth['adj_close'].isna())].head(10)

,Unnamed: 0,Date,adj_close,close,low,high,open,volume
197,197,2017-11-09,320.884003,320.884003,307.056000,329.451996,308.644989,8.932500e+08
198,198,2017-11-10,299.252991,299.252991,294.541992,324.717987,320.670990,8.859860e+08
199,199,2017-11-11,314.681000,314.681000,298.191986,319.453003,298.585999,8.423010e+08
200,200,2017-11-12,307.907990,307.907990,298.513000,319.153015,314.690002,1.613480e+09
201,201,2017-11-13,316.716003,316.716003,307.024994,328.415009,307.024994,1.041890e+09
202,202,2017-11-14,337.631012,337.631012,316.763000,340.177002,316.763000,1.069680e+09
203,203,2017-11-15,333.356995,333.356995,329.812988,340.911987,337.963989,7.226660e+08
204,204,2017-11-16,330.924011,330.924011,323.605988,336.158997,333.442993,7.972540e+08
205,205,2017-11-17,332.394012,332.394012,327.523010,334.963989,330.166992,6.217330e+08
206,206,2017-11-18,347.612000,347.612000,327.687012,349.615997,331.980011,6.496390e+08


In [6]:
df_eth.drop('Unnamed: 0', axis=1, inplace=True)
df_eth.drop('adj_close', axis=1, inplace=True)
df_eth.set_index('Date', inplace=True)
df_eth.index = df_eth.index.astype('datetime64[ns]')
df_eth.dropna(inplace=True)

In [7]:
df_eth

,close,low,high,open,volume
Date,,,,,
2017-11-09,320.884003,307.056000,329.451996,308.644989,8.932500e+08
2017-11-10,299.252991,294.541992,324.717987,320.670990,8.859860e+08
2017-11-11,314.681000,298.191986,319.453003,298.585999,8.423010e+08
2017-11-12,307.907990,298.513000,319.153015,314.690002,1.613480e+09
2017-11-13,316.716003,307.024994,328.415009,307.024994,1.041890e+09
...,...,...,...,...,...
2022-04-21,2987.480713,2962.410400,3173.451416,3077.829346,2.078359e+10
2022-04-22,2964.835693,2942.358643,3024.854492,2986.938721,1.678280e+10
2022-04-23,2938.114014,2926.740234,2975.322754,2964.802246,9.116956e+09


## Moving Average

In [8]:
eth_ma = df_eth['close'].to_frame()
eth_ma['SMA30'] = eth_ma['close'].rolling(30).mean()
eth_ma['CMA30'] = eth_ma['close'].expanding().mean()
eth_ma['EMA30'] = tb.EMA(df_eth['close'], timeperiod=30)
eth_ma.dropna(inplace=True)
eth_ma

,close,SMA30,CMA30,EMA30
Date,,,,
2017-12-08,456.031006,400.082003,400.082003,400.082003
2017-12-09,473.502014,405.169270,402.450390,404.818778
2017-12-10,441.721008,409.918204,403.677597,407.199567
2017-12-11,515.135986,416.600037,407.055124,414.163207
2017-12-12,651.431030,428.050805,414.242651,429.470808
...,...,...,...,...
2022-04-21,2987.480713,3199.351164,1063.847074,3095.944448
2022-04-22,2964.835693,3197.143449,1065.016194,3087.485819
2022-04-23,2938.114014,3191.478516,1066.167452,3077.848928


In [9]:
fig = px.line(eth_ma, x=eth_ma.index, y =['close', 'SMA30', 'CMA30', 'EMA30'],  color_discrete_sequence=px.colors.qualitative.G10)
fig

## Stochastic Oscillator

https://www.investopedia.com/terms/s/stochasticoscillator.asp

1. A stochastic oscillator is a popular technical indicator for generating overbought and oversold signals.
2. Buy/Sell when both K and D are below 20% or above 80%
3. Stochastic oscillators tend to vary around some mean price level, since they rely on an asset's price history.

In [10]:
slowk, slowd = tb.STOCH(df_eth["high"], df_eth["low"], df_eth["close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
eth_stoch = pd.DataFrame(index=df_eth.index,
                            data={"slowk": slowk,
                                  "slowd": slowd})

eth_stoch.dropna(inplace=True)


eth_stoch

,slowk,slowd
Date,,
2017-11-17,77.873856,80.783803
2017-11-18,81.735528,81.334123
2017-11-19,77.770765,79.126716
2017-11-20,82.435804,80.647365
2017-11-21,75.518558,78.575042
...,...,...
2022-04-21,64.679521,64.653946
2022-04-22,42.828174,61.069701
2022-04-23,21.151273,42.886322


In [11]:
def signal(k, d): 
    if (d <= 20 and k<=20) or (d >=80 and k>=80): 
        return 1
    else: return 0

In [12]:
eth_stoch['stoch_signal'] = eth_stoch.apply(lambda x: signal(x.slowk, x.slowd), axis=1)

In [13]:
fig = px.bar(eth_stoch, x=eth_stoch.index, y =['stoch_signal'],  color_discrete_sequence=px.colors.qualitative.G10)
fig

In [14]:
# for later use in the concat
stoch_c = ['slowk', 'slowd']

## MACD - Moving Average Convergence/Divergence

https://www.investopedia.com/terms/m/macd.asp

1. MACD triggers technical signals when it crosses above (to buy) or below (to sell) its signal line.
2. The speed of crossovers is also taken as a signal of a market is overbought or oversold.
3. MACD helps investors understand whether the bullish or bearish movement in the price is strengthening or weakening.

In [15]:
macd, macdsignal, macdhist = tb.MACD(df_eth.close, fastperiod=12, slowperiod=26, signalperiod=9)

In [16]:
eth_macd = pd.DataFrame(index=df_eth.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})

eth_macd.dropna(inplace=True)
eth_macd


,macd,macdsignal,macdhist
Date,,,
2017-12-12,53.550952,50.810936,2.740015
2017-12-13,66.384639,53.925677,12.458962
2017-12-14,75.128503,58.166242,16.962261
2017-12-15,80.216096,62.576213,17.639883
2017-12-16,84.226156,66.906201,17.319955
...,...,...,...
2022-04-21,-21.003927,5.781681,-26.785609
2022-04-22,-28.661926,-1.107040,-27.554886
2022-04-23,-36.466794,-8.178991,-28.287803


In [17]:
# for later use in the concat
macd_c = ['macd', 'macdsignal', 'macdhist']

## Bollinger Bands

In [18]:
upper, middle, lower = tb.BBANDS(df_eth["close"], timeperiod=30)
eth_bands = pd.DataFrame(index=df_eth.index,
                            data={"bb_low": lower,
                                  "bb_ma": middle,
                                  "bb_high": upper})
eth_bands.dropna(inplace=True)
eth_bands

,bb_low,bb_ma,bb_high
Date,,,
2017-12-08,272.823019,400.082003,527.340986
2017-12-09,278.781964,405.169270,531.556575
2017-12-10,289.228888,409.918204,530.607520
2017-12-11,295.546009,416.600037,537.654065
2017-12-12,286.958109,428.050805,569.143500
...,...,...,...
2022-04-21,2866.562736,3199.351164,3532.139591
2022-04-22,2859.081758,3197.143449,3535.205140
2022-04-23,2842.128490,3191.478516,3540.828542


In [19]:
# for later use in the concat
bands_c = ['bb_low', 'bb_ma', 'bb_high']

## Relative strength index (RSI)

https://www.investopedia.com/terms/r/rsi.asp

1. The RSI provides technical traders with signals about bullish and bearish price momentum, and it is often plotted beneath the graph of an asset’s price.
2. An asset is usually considered overbought when the RSI is above 70% and oversold when it is below 30%.

In [20]:
rsi = tb.RSI(df_eth['close'], timeperiod=30)

In [21]:
eth_rsi = pd.DataFrame(index=df_eth.index,
                            data={"close": df_eth['close'],
                                  "rsi": rsi})

eth_rsi.dropna(inplace=True)
eth_rsi.head(10)

,close,rsi
Date,,
2017-12-09,473.502014,67.039114
2017-12-10,441.721008,62.454273
2017-12-11,515.135986,67.728465
2017-12-12,651.431030,74.584993
2017-12-13,702.767029,76.528106
2017-12-14,695.815979,75.717230
2017-12-15,684.447998,74.383808
2017-12-16,696.208984,74.857681
2017-12-17,719.974976,75.793755


In [22]:
fig = px.line(eth_rsi, x=eth_rsi.index, y =['close', 'rsi'],  color_discrete_sequence=px.colors.qualitative.G10)
fig

## Fibonacci retracement

## Ichimoku cloud

## Standard deviation

In [23]:
stdev = tb.STDDEV(df_eth['close'], timeperiod=30, nbdev=1)

In [24]:
eth_stdev = pd.DataFrame(index=df_eth.index,
                            data={"close": df_eth['close'],
                                  "stdev": stdev})

eth_stdev.dropna(inplace=True)
eth_stdev


,close,stdev
Date,,
2017-12-08,456.031006,63.629492
2017-12-09,473.502014,63.193653
2017-12-10,441.721008,60.344658
2017-12-11,515.135986,60.527014
2017-12-12,651.431030,70.546348
...,...,...
2022-04-21,2987.480713,166.394214
2022-04-22,2964.835693,169.030846
2022-04-23,2938.114014,174.675013


## Average directional index

https://www.investopedia.com/terms/a/adx.asp#:~:text=The%20average%20directional%20index%20(ADX)%20is%20a%20technical%20analysis%20indicator,positive%20directional%20indicator%20(%2BDI).

1. The ADX makes use of a positive (+DI) and negative (-DI) directional indicator in addition to the trendline.
2. The trend has strength when ADX is above 25; the trend is weak or the price is trendless when ADX is below 20, according to Wilder.
3. Non-trending doesn't mean the price isn't moving. It may not be, but the price could also be making a trend change or is too volatile for a clear direction to be present.

In [25]:
adx = tb.ADX(df_eth['high'], df_eth['low'], df_eth['close'], timeperiod=20)

In [26]:
eth_adx = pd.DataFrame(index=df_eth.index,
                            data={"close": df_eth['close'],
                                  "adx": adx})

eth_adx.dropna(inplace=True)
eth_adx

,close,adx
Date,,
2017-12-18,794.645020,37.573268
2017-12-19,826.822998,38.843567
2017-12-20,819.085999,39.647736
2017-12-21,821.062988,40.545057
2017-12-22,674.859985,39.001209
...,...,...
2022-04-21,2987.480713,13.298290
2022-04-22,2964.835693,13.319277
2022-04-23,2938.114014,13.400342


## Creating full dataframe 

First, know the most recent time table present

In [27]:
df_dollar['dollar_close']

Series([], Name: dollar_close, dtype: float64)

In [28]:
result =pd.concat([df_eth, eth_ma[['SMA30','CMA30','EMA30']], eth_adx['adx'], eth_bands[bands_c], eth_macd[macd_c], eth_rsi['rsi'], eth_stdev['stdev'], eth_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)

In [29]:
result.dropna(inplace=True)

In [30]:
result

,close,low,high,open,volume,SMA30,CMA30,EMA30,adx,bb_low,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
